In [1]:
from google.colab import drive 
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import pandas as pd

df_temp=pd.read_csv('gdrive/MyDrive/Data/GlobalLandTemperaturesByCountry.csv') #Temperature df
df_co2=pd.read_csv('gdrive/MyDrive/Data/CO2emissionbycountries.csv',encoding= 'unicode_escape') #CO2 df

#df_t=pd.read_csv('gdrive/MyDrive/Data/city_temperature.csv') 

In [ ]:
df_temp

In [4]:
df_co2

,Country,Code,Calling Code,Year,CO2 emission (Tons),Population(2022),Area,% of World,Density(km2)
0,Afghanistan,AF,93,1750,0.0,41128771.0,652230.0,0.40%,63/km²
1,Afghanistan,AF,93,1751,0.0,41128771.0,652230.0,0.40%,63/km²
2,Afghanistan,AF,93,1752,0.0,41128771.0,652230.0,0.40%,63/km²
3,Afghanistan,AF,93,1753,0.0,41128771.0,652230.0,0.40%,63/km²
4,Afghanistan,AF,93,1754,0.0,41128771.0,652230.0,0.40%,63/km²
...,...,...,...,...,...,...,...,...,...
59615,Zimbabwe,ZW,263,2016,736467042.0,16320537.0,390757.0,0.30%,42/km²
59616,Zimbabwe,ZW,263,2017,746048675.0,16320537.0,390757.0,0.30%,42/km²
59617,Zimbabwe,ZW,263,2018,757903042.0,16320537.0,390757.0,0.30%,42/km²
59618,Zimbabwe,ZW,263,2019,768852126.0,16320537.0,390757.0,0.30%,42/km²


In [5]:
from datetime import datetime
df_temp['dt']= pd.to_datetime(df_temp['dt'])

df_temp['Year'] = pd.DatetimeIndex(df_temp['dt']).year
df_temp = df_temp.drop(columns=['dt','AverageTemperatureUncertainty'])
df_temp = df_temp[['Year', 'Country', 'AverageTemperature']]

df_temp = df_temp[df_temp['Year'] >= 1950]
df_temp = df_temp.reset_index()
print(df_temp)

         index  Year   Country  AverageTemperature
0         2474  1950     Åland              -5.611
1         2475  1950     Åland              -1.804
2         2476  1950     Åland               0.090
3         2477  1950     Åland               3.630
4         2478  1950     Åland               8.106
...        ...   ...       ...                 ...
185889  577457  2013  Zimbabwe              19.059
185890  577458  2013  Zimbabwe              17.613
185891  577459  2013  Zimbabwe              17.000
185892  577460  2013  Zimbabwe              19.759
185893  577461  2013  Zimbabwe                 NaN

[185894 rows x 4 columns]


In [6]:
df_temp2=df_temp.fillna(method='ffill')
df_temp2

,index,Year,Country,AverageTemperature
0,2474,1950,Åland,-5.611
1,2475,1950,Åland,-1.804
2,2476,1950,Åland,0.090
3,2477,1950,Åland,3.630
4,2478,1950,Åland,8.106
...,...,...,...,...
185889,577457,2013,Zimbabwe,19.059
185890,577458,2013,Zimbabwe,17.613
185891,577459,2013,Zimbabwe,17.000
185892,577460,2013,Zimbabwe,19.759


In [7]:
result = df_temp2.groupby(['Country','Year'])['AverageTemperature'].mean()

# Display result
type(result)
df_temp3 = result
df_temp3.to_frame().reset_index()
#df_temp3['AverageTemperature'].round(2)
df_temp3

Country      Year
Afghanistan  1950    13.043500
             1951    13.967750
             1952    14.175417
             1953    14.650750
             1954    13.691333
                       ...    
Åland        2009     6.489083
             2010     4.861917
             2011     7.170750
             2012     6.063917
             2013     7.362556
Name: AverageTemperature, Length: 15552, dtype: float64

In [ ]:
df_co2.head(5)

In [8]:
#Dropping data with CO2 emission equal 0
#df= df_co2[df_co2['CO2 emission (Tons)'] != 0]
#11880 
df = df_co2
df = df[df['Year'] >= 1959]

df = df[df['Year'] < 2014]

df.fillna(method='ffill')

df=df.drop(columns=['Code','Calling Code','Area','% of World','Density(km2)'])
df = df.rename(columns={'CO2 emission (Tons)': 'CO2'})
df

,Country,Year,CO2,Population(2022)
209,Afghanistan,1959,1839179.0,41128771.0
210,Afghanistan,1960,2253064.0,41128771.0
211,Afghanistan,1961,2743862.0,41128771.0
212,Afghanistan,1962,3432456.0,41128771.0
213,Afghanistan,1963,4139192.0,41128771.0
...,...,...,...,...
59608,Zimbabwe,2009,664256525.0,16320537.0
59609,Zimbabwe,2010,672134841.0,16320537.0
59610,Zimbabwe,2011,681878824.0,16320537.0
59611,Zimbabwe,2012,689761713.0,16320537.0


In [9]:
#Left outer join exclusion on country with data surveys starting from 1959
df_final=pd.merge(df,df_temp3,on=['Country','Year'],how="inner",indicator=True)
#df_final=df_final[df_final['_merge']=='left_only']
# resetting the DataFrame index
df_final = df_final.reset_index()

df_final=df_final.drop(columns=['index','_merge'])
df_final = df_final.round(2)
df_final['CO2'] = df_final['CO2'].astype(int)
print(df_final)
df_final.to_csv('data_prepared.csv')
!cp data_prepared.csv "gdrive/MyDrive/Data"